# [STARTER] Exercise - Create an Agent with external API call enabled

In this exercise, you'll build an agent that can interact with external APIs to gather real-time data
and provide responses based on that information. You'll combine concepts from state management and
memory while adding the ability to make external API calls safely and effectively.


## Challenge

Your task is to create an agent that can make External API Calls:

- Implement tools that interact with real APIs
- Handle API responses and errors gracefully
- Use environment variables for API keys
- Process and format API data for user consumption

## Setup
First, let's import the necessary libraries:

In [28]:
import os
import random
from typing import List
import requests
from dotenv import load_dotenv

from lib.agents import Agent
from lib.messages import BaseMessage
from lib.tooling import tool

In [29]:
load_dotenv(override=True)

True

## Define API tools

Feel free to use any open service available through APIs.

Here are a few examples. You can follow the instructions given.
- https://jsonplaceholder.typicode.com/guide/
- https://www.exchangerate-api.com/
- https://openweathermap.org/

Or you can find one you're interested in here.
- https://github.com/public-apis/public-apis

In [30]:
@tool
def get_exchange_rate(from_currency: str = "USD") -> dict:
    """
    Get latest exchange rates from a base currency
    args:
        from_currency (str): Base currency code (default: USD)
    """
    API_KEY = os.getenv("EXCHANGERATE_API_KEY")
    BASE_URL = "https://v6.exchangerate-api.com/v6"
    
    url = f"{BASE_URL}/{API_KEY}/latest/{from_currency}"
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

In [31]:
@tool
def get_weather(city: str) -> dict:
    """
    Get current weather for a city using OpenWeather API
    args:
        city (str): Name of the city to get weather for
    """
    API_KEY = os.getenv("OPENWEATHER_API_KEY")
    BASE_URL = "https://api.openweathermap.org/data/2.5/weather"
    
    params = {
        "q": city,
        "appid": API_KEY,
        "units": "metric"
    }
    
    response = requests.get(BASE_URL, params=params)
    response.raise_for_status()
    return response.json()

In [22]:
# TODO: Define as many tools that access external APIs as you want
# Example:
@tool
def get_random_pokemon() -> dict:
    """Get a random Pokemon from the original 151"""
    URL = "https://pokeapi.co/api/v2/pokemon?limit=151"
    response = requests.get(URL)
    response.raise_for_status()
    return random.choice(response.json()['results'])

In [23]:
# TODO: Add all the tools you have defined
tools = [get_random_pokemon, get_weather, get_exchange_rate]

In [24]:
# TODO: Add instructions to your agent

agent = Agent(
    model_name="gpt-4o-mini",
    instructions=(
        "You are an assistant that can help with:\n"
        "1. Getting weather information for cities\n"
        "2. Getting exchange rates for currencies\n"
        "3. Getting a random pokemon"
        "Use the available tools to help answer questions about these topics."
        "Maintain context across conversations within the same session."
    ),
    tools=tools
)

In [25]:
def print_messages(messages: List[BaseMessage]):
    for m in messages:
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

## Run your Agent

In [33]:
# TODO: Change the query and then run your agent
query = "Give me a pokemon"
session_id = "external_tools"

In [34]:
run1 = agent.invoke(
    query=query, 
    session_id=session_id,
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are an assistant that can help with:
1. Getting weather information for cities
2. Getting exchange rates for currencies
3. Getting a random pokemonUse the available tools to help answer questions about these topics.Maintain context across conversations within the same session., tool_calls = None)
 -> (role = user, content = Give me a pokemon, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageFunctionToolCall(id='call_WtLSPoPToYHkNL2FWfE3fDMw', function=Function(arguments='{}', name='get_random_pokemon'), type='function')])
 -> (role = tool, content = "{'name': 'nidoran-m', 'url': 'https://pokeapi.co/api/v2/pokemon/32/'}", tool_calls = 

## Check session histories

In [35]:
runs = agent.get_session_runs(session_id)
for i, run_object in enumerate(runs, 1):
    print(f"\n# Run {i}", run_object.metadata)
    print("Messages:")
    print_messages(run_object.get_final_state()["messages"])


# Run 1 {'run_id': 'c2406dea-bcb9-4526-a9f8-38b7b0721573', 'start_timestamp': '2025-12-29 22:10:59.724920', 'end_timestamp': '2025-12-29 22:11:03.057808', 'snapshot_counts': 5}
Messages:
 -> (role = system, content = You are an assistant that can help with:
1. Getting weather information for cities
2. Getting exchange rates for currencies
3. Getting a random pokemonUse the available tools to help answer questions about these topics.Maintain context across conversations within the same session., tool_calls = None)
 -> (role = user, content = Give me a pokemon, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageFunctionToolCall(id='call_WtLSPoPToYHkNL2FWfE3fDMw', function=Function(arguments='{}', name='get_random_pokemon'), type='function')])
 -> (role = tool, content = "{'name': 'nidoran-m', 'url': 'https://pokeapi.co/api/v2/pokemon/32/'}", tool_calls = None)
 -> (role = assistant, content = You got a Pokémon: **Nidoran♂**! 

You can find more 

In [36]:
runs = agent.get_session_runs(session_id)
for run_object in runs:
    print(run_object)
    for snp in run_object.snapshots:
        print(f"-> {snp}")
    print("\n")

Run('c2406dea-bcb9-4526-a9f8-38b7b0721573')
-> Snapshot('6e3f7e57-2287-4634-959b-5faa1fa9bd88') @ [2025-12-29 22:10:59.725099]: __entry__.State({'user_query': 'Give me a pokemon', 'instructions': 'You are an assistant that can help with:\n1. Getting weather information for cities\n2. Getting exchange rates for currencies\n3. Getting a random pokemonUse the available tools to help answer questions about these topics.Maintain context across conversations within the same session.', 'messages': [], 'current_tool_calls': None, 'session_id': 'external_tools'})
-> Snapshot('2d7693b8-526b-4978-bbbd-946f112689a7') @ [2025-12-29 22:10:59.725280]: message_prep.State({'user_query': 'Give me a pokemon', 'instructions': 'You are an assistant that can help with:\n1. Getting weather information for cities\n2. Getting exchange rates for currencies\n3. Getting a random pokemonUse the available tools to help answer questions about these topics.Maintain context across conversations within the same session